In [1]:
from pathlib import Path
import pandas as pd
import mne

# ------------------
# Paths & I/O
# ------------------
LOCALIZER_EPO = Path("/Users/Barbu/Documents/Postdoc/Experiments/Pushmi/MEG/Data/Pilot/BIDS/derivatives/mne-bids-pipeline/sub-01/meg/sub-01_task-localizer_epo.fif")
BEHAVIOR_XPD = Path("/Users/Barbu/Documents/Postdoc/Experiments/Pushmi/MEG/Data/Pilot/Behavior/sub-01/data.xpd")

epochs = mne.read_epochs(LOCALIZER_EPO)
meta = epochs.metadata.reset_index(drop=True)


Reading /Users/Barbu/Documents/Postdoc/Experiments/Pushmi/MEG/Data/Pilot/BIDS/derivatives/mne-bids-pipeline/sub-01/meg/sub-01_task-localizer_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    1000.00 ms
        0 CTF compensation matrices available
Adding metadata with 3 columns
960 matching events found
No baseline correction applied
0 projection items activated


In [6]:
# ------------------
# Behavioral dataset
# ------------------
behavior = (
    pd.read_csv(BEHAVIOR_XPD, comment="#")
      .query("trial_type == 'localizer' and trial_block != 0")
      .assign(
          subject_id = 1,
          trial_number = lambda d: d.trial_number - 64
      )
      .loc[:, [
          "subject_id", "trial_type", "trial_block", "trial_number",
          "shape1", "label1", "correct_key",
          "participant_response", "reaction_time", "participant_correct"
      ]]
      .reset_index(drop=True)
)

# ------------------
# Align & merge by row index
# ------------------
assert len(behavior) == len(meta), f"Row mismatch: behavior={len(behavior)} vs meta={len(meta)}"

merged = (
    behavior
      .join(meta, rsuffix="_meta")
      .rename(columns={"trial_type": "task"})
      .assign(stimulus=lambda d: d["shape1"].fillna(d["label1"]))
)

merged = merged.replace({"carré": "carre", "losenge": "losange"}, regex=True)

epochs.metadata = merged

# words = ["biche", "dinde"]
# epochs[f"stimulus in {words}"].plot(n_channels=29, events=True)

Replacing existing metadata with 14 columns


In [7]:
merged.to_csv("metadata_localizer.csv", index=False)